In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk
from tkinter import filedialog

sns.set()

In [3]:
def select_audio_file():
    root = tk.Tk()
    root.withdraw()  
 
    audio_file_path = filedialog.askopenfilename(
        title="Select an audio file",
        filetypes=[("Audio Files", "*.mp3 *.wav *.flac *.ogg *.m4a")]
    )

    if audio_file_path:
        return audio_file_path
    else:
        return None

In [4]:
selected_audio_file = select_audio_file()

if selected_audio_file:
    print(f"Selected audio file: {selected_audio_file}")
else:
    print("No audio file selected.")

In [4]:
audio_file_path = filedialog.askopenfilename(
    title="Select an audio file",
    filetypes=[("Audio Files", "*.mp3 *.wav *.flac *.ogg *.m4a")]
)